In [6]:
import json

In [18]:
with open("trec-covid.jsonl") as f:
    results = [json.loads(i) for i in f.readlines()]

In [8]:
results[0]

{'_id': '1',
 'text': 'what is the origin of COVID-19',
 'metadata': {'query': 'coronavirus origin',
  'narrative': "seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"},
 'question': 'what is the origin of COVID-19',
 'retrieval': [{'source': '/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/corpus.jsonl',
   'seq_num': 129058,
   'document': 'The current pandemic of COVID-19 has generated many challenging questions for the scientific community, ranging from queries about the origin of the virus to its pathogenesis and clinical management. This article is protected by copyright. All rights reserved.'},
  {'source': '/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/corpus.jsonl',
   'seq_num': 90806,
   'document': "Since the identification of the first cases of the coronavirus in December 2019 in Wuhan, China, there has been a significant amount of confusion regarding 

In [9]:
with open("/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/corpus.jsonl") as f:
    corpus = [json.loads(i) for i in f.readlines()]
    corpus_dict = {}
    for c in corpus:
        corpus_dict[c["_id"]] = c["text"]

In [10]:
seq_num_to_id = {}
for i, doc in zip(range(1, len(corpus) + 1), corpus):
    seq_num_to_id[i] = doc["_id"] 

In [19]:
# Add document id to each retrieved document
for res in results:
    for retrieval in res["retrieval"]:
        seq_num = retrieval["seq_num"]
        retrieval["_id"] = seq_num_to_id[seq_num]

In [22]:
# Get retrieval results as corpus id
retrieval_results = {}
for res in results:
    corpus_ids = []
    for retrieval in res["retrieval"]:
        corpus_ids.append(retrieval["_id"])
    retrieval_results[res["_id"]] = corpus_ids

In [23]:
retrieval_results

{'1': ['n15i01tn',
  'hmvo5b0q',
  'dv9m19yk',
  'm4y8tf6u',
  'utsr0zv7',
  'k9lcpjyo',
  'jbtrdvhe',
  'r2ynbnxx',
  'jxbk30gh',
  'rtw8m00s',
  'juz9jnfk',
  'lj2iu7z0',
  't1iagum7',
  '4dtk1kyh',
  'jwxt4ygt',
  'wco27nop',
  'dnxhtbxn',
  '8qfwat2v',
  'deee71uw',
  'gjigvjgv',
  'xndojxqk',
  'ccxj4s6j',
  'rugijb30',
  'p3681yit',
  '2lxs9laj',
  '1ybj2p1n',
  '80c471y3',
  'w3avnbpu',
  'fsy00ngb',
  '75773gwg',
  'ptizke03',
  'bbae1qyx',
  'k9yus2sv',
  'qp4efhwq',
  '7v5aln90',
  '6ck2ntid',
  '8a6flxl6',
  'f6p9i951',
  '8vi60e0a',
  'can1e8ro',
  '82sc928x',
  'dtv7to3l',
  'hqzkzupi',
  'sqrn6kjy',
  '9griuata',
  'pfv7q4v6',
  '7qtlj9t8',
  'i14gkdw0',
  '1x0tvnho',
  'nph01vl6',
  '0nh58odf',
  'drzk1uq6',
  'nk7w2ola',
  'tfkhptdg',
  'p3gswh23',
  '475nei28',
  'u7tz6xzz',
  'aura0th0',
  'v9ftmdlv',
  'okmpsv47',
  'uv3djven',
  '6k5ac3f2',
  'gmxhmzki',
  'jb6o4v6p',
  'sigbm70i',
  '431buxxr',
  'xibub2in',
  'ectly1v4',
  'm0vqst1q',
  'c2bdmgj7',
  'sajlyf4t',
 

## Load Qrels

In [25]:
import pandas as pd
qrels = pd.read_csv("/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/qrels/test.tsv", sep="\t")

In [26]:
qrels.head(10)

,query-id,corpus-id,score
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1
5,1,02f0opkr,1
6,1,047xpt2c,0
7,1,04ftw7k9,0
8,1,pl9ht0d0,0
9,1,05vx82oo,0


In [55]:
import numpy as np
from sklearn.metrics import ndcg_score
scores = {}
for qid, dids in retrieval_results.items():
    qrels_ = qrels[qrels["query-id"]==int(qid)]
    qrels_.loc[:, "ret"] = 0
    qrels_.loc[qrels_["corpus-id"].isin(dids), "ret"] = 1
    y_true = qrels_["score"].values
    y_pred = qrels_["ret"].values
    score = ndcg_score([y_true], [y_pred], k=100)
    scores[qid] = round(score * 100, 2)

/tmp/ipykernel_1383847/1972854865.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qrels_.loc[:, "ret"] = 0
/tmp/ipykernel_1383847/1972854865.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qrels_.loc[:, "ret"] = 0
/tmp/ipykernel_1383847/1972854865.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

ValueError: ndcg_score should not be used on negative y_true values.

In [62]:
min(scores, key=scores.get)
scores["19"], results[19]

(22.08,
 {'_id': '20',
  'text': 'are patients taking Angiotensin-converting enzyme inhibitors (ACE) at increased risk for COVID-19?',
  'metadata': {'query': 'coronavirus and ACE inhibitors',
   'narrative': 'Looking for information on interactions between  coronavirus and  angiotensin converting enzyme 2 (ACE2) receptors, risk for patients taking these medications, and recommendations for these patients.'},
  'question': 'are patients taking Angiotensin-converting enzyme inhibitors (ACE) at increased risk for COVID-19?',
  'retrieval': [{'source': '/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/corpus.jsonl',
    'seq_num': 111221,
    'document': 'There has been a lot of speculation that patients with coronavirus disease 2019 (COVID-19) who are receiving angiotensin-converting enzyme (ACE) inhibitors or angiotensin receptor blockers (ARBs) may be at increased risk for adverse outcomes We reviewed the available evidence, and have not found this to be the case We recom

In [61]:
max(scores, key=scores.get)
scores["20"], results[20]

(93.0,
 {'_id': '21',
  'text': 'what are the mortality rates overall and in specific populations',
  'metadata': {'query': 'coronavirus mortality',
   'narrative': 'Seeking  information on COVID-19 fatality rates in different countries and in different population groups based on gender, blood types, or other factors'},
  'question': 'what are the mortality rates overall and in specific populations',
  'retrieval': [{'source': '/gallery_louvre/dayoon.ko/research/sds/src/datasets/trec-covid/corpus.jsonl',
    'seq_num': 41725,
    'document': 'BACKGROUND Detailed assessments of mortality patterns, particularly age-specific mortality, represent a crucial input that enables health systems to target interventions to specific populations. Understanding how all-cause mortality has changed with respect to development status can identify exemplars for best practice. To accomplish this, the Global Burden of Diseases, Injuries, and Risk Factors Study 2016 (GBD 2016) estimated age-specific and se

# Document Length

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, DirectoryLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import fire
import torch
from tqdm import tqdm
from glob import glob 

data_dir = "/gallery_louvre/dayoon.ko/research/sds/src/datasets"
dataset_name = "nq"
glob_dir = "corpus.jsonl"
loader = JSONLoader(f"{data_dir}/{dataset_name}/{glob_dir}", jq_schema=".text", json_lines=True)
documents = loader.load()
print(f'Document count: {len(documents)}')

# Split document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=10)
splits = text_splitter.split_documents(documents)
print(len(splits))

Document count: 2681468


KeyboardInterrupt: 

In [ ]:
# hotpotqa: 5233329
# trec-covid: 173???
# np: 2681468